In [21]:
import numpy as np

from strat import MACDandEMA
from datetime import datetime

from quantfreedom.enums import *
from quantfreedom.helper_funcs import dl_ex_candles
from quantfreedom.simulate import run_df_backtest, or_backtest


np.set_printoptions(formatter={"float_kind": "{:0.2f}".format})

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [22]:
candles = dl_ex_candles(
    exchange="mufex",
    symbol="BTCUSDT",
    timeframe="5m",
    candles_to_dl=6000,
)

In [24]:
macd_strat = MACDandEMA(
    long_short="long",
    ema_length=np.array([150, 300]),
    fast_length=np.array([10,20]),
    macd_below=np.array([200, 400]),
    signal_smoothing=np.array([7, 15]),
    slow_length=np.array([30, 50]),
)

In [25]:
backtest_settings = BacktestSettings(gains_pct_filter=10)

exchange_settings = ExchangeSettings(
    asset_tick_step=3,
    leverage_mode=1,
    leverage_tick_step=2,
    limit_fee_pct=0.0003,
    market_fee_pct=0.0006,
    max_asset_size=100.0,
    max_leverage=150.0,
    min_asset_size=0.001,
    min_leverage=1.0,
    mmr_pct=0.004,
    position_mode=3,
    price_tick_step=1,
)

In [26]:
static_os = StaticOrderSettings(
    increase_position_type=IncreasePositionType.RiskPctAccountEntrySize,
    leverage_strategy_type=LeverageStrategyType.Dynamic,
    pg_min_max_sl_bcb="min",
    sl_strategy_type=StopLossStrategyType.SLBasedOnCandleBody,
    sl_to_be_bool=False,
    starting_bar=50,
    starting_equity=1000.0,
    static_leverage=None,
    tp_fee_type="limit",
    tp_strategy_type=TakeProfitStrategyType.RiskReward,
    trail_sl_bool=True,
    z_or_e_type=None,
)

In [27]:
dos_arrays = DynamicOrderSettingsArrays(
    max_equity_risk_pct=np.array([12]),
    max_trades=np.array([0]),
    risk_account_pct_size=np.array([3]),
    risk_reward=np.array([5]),
    sl_based_on_add_pct=np.array([0.1, 0.25]),
    sl_based_on_lookback=np.array([20, 50]),
    sl_bcb_type=np.array([CandleBodyType.Low]),
    sl_to_be_cb_type=np.array([CandleBodyType.Nothing]),
    sl_to_be_when_pct=np.array([0]),
    trail_sl_bcb_type=np.array([CandleBodyType.Low]),
    trail_sl_by_pct=np.array([1, 2]),
    trail_sl_when_pct=np.array([1, 2]),
)

In [28]:
backtest_results = run_df_backtest(
    backtest_settings=backtest_settings,
    candles=candles,
    dos_arrays=dos_arrays,
    exchange_settings=exchange_settings,
    static_os=static_os,
    strategy=macd_strat,
)

Starting the backtest now ... and also here are some stats for your backtest.

Total indicator settings to test: 32
Total order settings to test: 16
Total combinations of settings to test: 512
Total candles: 6,000
Total candles to test: 3,072,000


In [34]:
backtest_results.sort_values(by=["qf_score"], ascending=False).head(10)

,ind_set_idx,dos_index,total_trades,wins,losses,gains_pct,win_rate,qf_score,fees_paid,ending_eq,total_pnl
95,5,15,17.0,8,9,389.108,47.059,0.649,388.168,4891.085,3891.085
333,20,13,18.0,9,9,336.137,50.000,0.645,365.609,4361.368,3361.368
77,4,13,18.0,9,9,198.428,50.000,0.642,253.053,2984.280,1984.280
351,21,15,18.0,8,10,410.742,44.444,0.640,399.867,5107.423,4107.423
397,24,13,17.0,8,9,271.737,47.059,0.592,301.309,3717.374,2717.374
461,28,13,17.0,8,9,271.737,47.059,0.592,301.309,3717.374,2717.374
365,22,13,17.0,8,9,286.202,47.059,0.583,300.581,3862.015,2862.015
31,1,15,16.0,7,9,358.532,43.750,0.582,374.797,4585.321,3585.321
78,4,14,18.0,8,10,404.766,44.444,0.578,419.706,5047.659,4047.659
79,4,15,18.0,8,10,404.766,44.444,0.578,419.706,5047.659,4047.659


In [33]:
order_records_df = or_backtest(
    backtest_settings=backtest_settings,
    candles=candles,
    dos_arrays=dos_arrays,
    exchange_settings=exchange_settings,
    static_os=static_os,
    strategy=macd_strat,
    ind_set_index=21,
    dos_index=15,
    plot_results=True,
    logger_bool=False,
)

DynamicOrderSettings(
    max_equity_risk_pct = 0.12,
    max_trades = 0,
    risk_account_pct_size = 0.03,
    risk_reward = 5.0,
    sl_based_on_add_pct = 0.0025,
    sl_based_on_lookback = 50,
    sl_bcb_type = 3,
    sl_to_be_cb_type = 6,
    sl_to_be_when_pct = 0.0,
    trail_sl_bcb_type = 3,
    trail_sl_by_pct = 0.02,
    trail_sl_when_pct = 0.02,
)


In [ ]:
order_records_df

,ind_set_idx,or_set_idx,bar_idx,timestamp,datetime,order_status,equity,available_balance,cash_borrowed,cash_used,...,entry_size_usd,entry_price,exit_price,position_size_asset,position_size_usd,realized_pnl,sl_pct,sl_price,tp_pct,tp_price
0,13,15,978,1707798600000,2024-02-13 04:30:00,EntryFilled,1000.000,948.282,4284.618,51.718,...,4336.336,49915.1,NaN,0.087,4336.336,NaN,0.600,49629.5,3.6,51687.2
1,13,15,1066,1707825000000,2024-02-13 11:50:00,EntryFilled,1000.000,855.388,10897.093,144.612,...,2420.751,49961.7,NaN,0.135,6757.087,NaN,0.800,49548.1,4.5,52189.7
2,13,15,1075,1707827700000,2024-02-13 12:35:00,EntryFilled,1000.000,715.081,20754.161,284.919,...,3384.900,49939.7,NaN,0.203,10141.987,NaN,0.800,49548.1,4.5,52191.5
3,13,15,1082,1707829800000,2024-02-13 13:10:00,EntryFilled,1000.000,527.983,33770.710,472.017,...,3346.579,49939.1,NaN,0.270,13488.566,NaN,0.800,49548.1,4.5,52198.9
4,13,15,1087,1707831300000,2024-02-13 13:35:00,StopLossFilled,880.177,880.177,NaN,NaN,...,NaN,NaN,49548.1,NaN,NaN,-119.823,NaN,NaN,NaN,NaN
5,13,15,1276,1707888000000,2024-02-14 05:20:00,EntryFilled,880.177,839.766,2819.852,40.411,...,2860.263,49488.2,NaN,0.058,2860.263,NaN,0.800,49097.5,4.6,51782.7
6,13,15,1320,1707901200000,2024-02-14 09:00:00,MovedTSL,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.014,50160.1,NaN,NaN
7,13,15,1321,1707901500000,2024-02-14 09:05:00,MovedTSL,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.016,50265.6,NaN,NaN
8,13,15,1322,1707901800000,2024-02-14 09:10:00,MovedTSL,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.020,50455.1,NaN,NaN
9,13,15,1324,1707902400000,2024-02-14 09:20:00,MovedTSL,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.024,50653.9,NaN,NaN
